In [ ]:
#https://keras.io/examples/timeseries/timeseries_transformer_classification/
#https://saturncloud.io/blog/understanding-keras-conv1d-layer-parameters-filters-and-kernel-size/#:~:text=Filters%20in%20Conv1D&text=In%20the%20context%20of%20Conv1D,pattern%20in%20the%20input%20data.
import math
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import plotly.graph_objects as go

df = pd.read_excel('Data2.xlsx', sheet_name=0)
ordered_df = df.iloc[::-1]

date_list = df.iloc[:, 0].values
price_list = df.iloc[:, 1].values
features_list = df.iloc[:, 1:16].values

#Scale data
scaler = StandardScaler()
scaled_data = scaler.fit_transform(features_list)
price_list = np.reshape(price_list, (price_list.shape[0], 1))
output_scaler = StandardScaler()
scaled_output_data = output_scaler.fit_transform(price_list)
#Train test split
training_data_len = math.ceil(len(scaled_data)* .85)
training_data_len

#Train test split
train_data = scaled_data[0:training_data_len, :]
x_train = []
y_train = []
look_back_window = 20

for i in range(look_back_window, len(train_data)):
    x_train.append(train_data[i-look_back_window:i])
    y_train.append(train_data[i, 0])
#Convert to np array
x_train, y_train = np.array(x_train), np.array(y_train)

test_data = scaled_data[training_data_len-look_back_window:, :]

x_test = []
y_test = price_list[training_data_len:]
y_test = np.array(y_test)
y_test = np.reshape(y_test, (y_test.shape[0], 1))

for i in range(look_back_window, len(test_data)):
    x_test.append(test_data[i-look_back_window:i])

x_test = np.array(x_test)

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Normalization and Attention
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(x, x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation=layers.LeakyReLU())(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res
def build_model(
    input_shape,
    head_size,
    num_heads,
    ff_dim,
    num_transformer_blocks,
    mlp_units,
    dropout=0,
    mlp_dropout=0,
):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(1)(x)
    return keras.Model(inputs, outputs)

In [ ]:
input_shape = x_train.shape[1:]

model = build_model(
    input_shape,
    head_size=64,
    num_heads=4,
    ff_dim=2,
    num_transformer_blocks=4,
    mlp_units=[64],
    mlp_dropout=0.3,
    dropout=0.3,
)
model.summary()
model.compile(optimizer='adam', loss="mean_squared_error")

callbacks = [keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)]

model.fit(
    x_train,
    y_train,
    validation_split=0.1,
    epochs=20,
    batch_size=16,
    callbacks=callbacks,
)
predictions = model.predict(x_test)
predictions = output_scaler.inverse_transform(predictions)
rmse = np.sqrt(np.mean(predictions - y_test)**2)
print(rmse)
train = np.squeeze(price_list[:training_data_len])
valid = np.squeeze(price_list[training_data_len:])
predictions = np.squeeze(predictions)
train_date_list = date_list[:training_data_len]
valid_date_list = date_list[training_data_len:]
valid_date_list2 = date_list[training_data_len:]
fig = go.Figure()
fig.add_trace(go.Scatter(x=train_date_list, y=train, name='train',  mode='lines'))
fig.add_trace(go.Scatter(x=valid_date_list, y=valid, name='valid',  mode='lines'))
fig.add_trace(go.Scatter(x=valid_date_list2, y=predictions, name='predict',  mode='lines'))
fig.show()

In [ ]:
input_shape = x_train.shape[1:]

model = build_model(
    input_shape,
    head_size=256,
    num_heads=4,
    ff_dim=8,
    num_transformer_blocks=10,
    mlp_units=[128],
    mlp_dropout=0.2,
    dropout=0.2,
)
model.summary()
model.compile(optimizer='adam', loss="mean_squared_error")

callbacks = [keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]

model.fit(
    x_train,
    y_train,
    validation_split=0.2,
    epochs=200,
    batch_size=32,
    callbacks=callbacks,
    shuffle=True
)
predictions = model.predict(x_test)
predictions = output_scaler.inverse_transform(predictions)
rmse = np.sqrt(np.mean(predictions - y_test)**2)
print(rmse)

In [ ]:
train = np.squeeze(price_list[:training_data_len])
valid = np.squeeze(price_list[training_data_len:])
predictions = np.squeeze(predictions)
train_date_list = date_list[:training_data_len]
valid_date_list = date_list[training_data_len:]
valid_date_list2 = date_list[training_data_len:]
fig = go.Figure()
fig.add_trace(go.Scatter(x=train_date_list, y=train, name='train',  mode='lines'))
fig.add_trace(go.Scatter(x=valid_date_list, y=valid, name='valid',  mode='lines'))
fig.add_trace(go.Scatter(x=valid_date_list2, y=predictions, name='predict',  mode='lines'))
fig.show()

In [ ]:
input_shape = x_train.shape[1:]

model = build_model(
    input_shape,
    head_size=512,
    num_heads=3,
    ff_dim=3,
    num_transformer_blocks=3,
    mlp_units=[256],
    mlp_dropout=0.2,
    dropout=0.2,
)
model.summary()
model.compile(optimizer='adam', loss="mean_squared_error")

callbacks = [keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)]

model.fit(
    x_train,
    y_train,
    validation_split=0.2,
    epochs=200,
    batch_size=32,
    callbacks=callbacks,
)
predictions = model.predict(x_test)
predictions = output_scaler.inverse_transform(predictions)
rmse = np.sqrt(np.mean(predictions - y_test)**2)
print(rmse)

In [ ]:
train = np.squeeze(price_list[:training_data_len])
valid = np.squeeze(price_list[training_data_len:])
predictions = np.squeeze(predictions)
train_date_list = date_list[:training_data_len]
valid_date_list = date_list[training_data_len:]
valid_date_list2 = date_list[training_data_len:]
fig = go.Figure()
fig.add_trace(go.Scatter(x=train_date_list, y=train, name='train',  mode='lines'))
fig.add_trace(go.Scatter(x=valid_date_list, y=valid, name='valid',  mode='lines'))
fig.add_trace(go.Scatter(x=valid_date_list2, y=predictions, name='predict',  mode='lines'))
fig.show()